In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
print('Versão do TensorFlow: ', tf.__version__)

In [ ]:
tf.config.list_physical_devices()

In [ ]:
# Carregar os dados do arquivo .pkl
with open('./data.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [ ]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 saídas: 0, 1, 2, 3, 4
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=32000, validation_split=0.2, callbacks=[tensorboard_callback])


In [ ]:
# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine

# GO TO RIGHT
targetX = 680
agentX = 340
angle = 0
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RoteteLeft', 'RotateRight', 'TurnOnEngine']
predictions = model.predict(np.array([[targetX, agentX, angle]]))
 
score = tf.nn.softmax(predictions[0])
print(
    "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# GO TO LEFT
targetX =340  
agentX = 680
angle = 0

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be GO TO LEFT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# ROTATE LEFT
targetX = 680
agentX = 340
angle = -30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be ROTATE LEFT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# ROTATE RIGHT
targetX = 680
agentX = 340
angle = -30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
score = tf.nn.softmax(predictions[0])
print( "{}-Must be ROTATE RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)



In [ ]:
%tensorboard

In [ ]:
model.save("./models/model_saved.h5")
model.save(filepath="./models/model_saved.keras")


 

### From here to you only MACOS to test 

In [ ]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("./models/model_saved.keras")

targetX = 0
agentX = 340
angle =  30

predictions = model.predict(np.array([[targetX, agentX, angle]]))
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RotateLeft
# 4 RotateRight
# 5 TurnOnEngine
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RotateLeft', 'RotateRight', 'TurnOnEngine']

score = tf.nn.softmax(predictions[0])
print(
    "{} This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('./models/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, './models/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model './models/saved_model/' models/web_model